In [ ]:
import czifile
import numpy as np

import bebi103

import holoviews as hv
hv.extension('bokeh')
import colorcet

import bokeh.io
bokeh.io.output_notebook()

In [ ]:
def dapi_sort(img):
    '''Reduces dimensions of dapi images to DAPI channel and z-stacks'''
    return img[0,0,0,0,:,:,:,0]

def image_reducer(filepath, zoom):
    '''Reduces the size of an inputed image'''
    # Read czi file
    img = czifile.imread(filepath)
    
    # Get rid of unnecessary dimensions
    img = dapi_sort(img)
    
    # Do a z-sum projection
    img = np.sum(img, 0)
    
    # Convert to float 
    img = (img.astype(float) - img.min()) / (img.max() - img.min())
    
    # Zoom if desired
    if zoom != False:
        return img[zoom]
    
    else:
        return img
    
def im_list(img):
    return bebi103.image.imshow(img,
                               frame_height = 100,
                               interpixel_distance = 0.65,
                               length_units = 'µm')

def im_splitter(filepath,
               crop_size = 5000,
               split_size = 1000):
    '''Imports a large image and splits it into an array of sub-images'''
    img = image_reducer(filepath, np.s_[:crop_size, :crop_size])
    
    init_array = np.arange(0, crop_size + split_size, split_size)
    
    seg_array = [np.s_[init_array[i]:init_array[i+1]] for i in range(len(init_array)-1)]
    
    seg_all = []

    for i in seg_array:
        for j in seg_array:
            seg_all.append(np.s_[i,j])
            
    return [img[i] for i in seg_all] 

In [ ]:
im3_ls = im_splitter('20.02.15_DAPI-seg-3.czi')

im3_ls[0].shape

In [ ]:
hv.Image(im3_ls[10]
).opts(cmap='viridis')

In [ ]:
ls = np.arange(0, 2000, 500)

ls_all = [np.s_[ls[i]:ls[i+1]] for i in range(len(ls)-1)]

ls

In [ ]:
ls_comp = []

for i in ls_all:
    for j in ls_all:
        ls_comp.append(np.s_[i,j])
        
len(ls_comp)

In [ ]:
im_split = [im1[i] for i in ls_comp]

im_split[0].shape

In [ ]:
im_sublist = [im_list(i) for i in im_split]

In [ ]:
len(im_sublist)

In [ ]:
bokeh.io.show(bokeh.layouts.gridplot(im_sublist, ncols = 3))